In [19]:
## por rapidez y comodidad vuelvo a cargarme todas las librerias que tengo en los demás notebooks, mas tensorflow


import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

import re
import os

from scipy.stats import iqr   ### para función que quita los outliers

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedKFold, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix , r2_score, mean_absolute_error, mean_squared_error    

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance
from sklearn.naive_bayes import GaussianNB
import multiprocessing
from sklearn.svm import SVC
import xgboost

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer


## PARA LAS TRANSFORMACIONES
from scipy.stats import shapiro
from scipy.stats import skew
from scipy import stats, special

#para guardar el modelo de ML
import pickle

import tensorflow as tf

import warnings
warnings.filterwarnings('once')

In [20]:
## cargo el dataframe
df_descr = pd.read_csv("..\\data\\processed\\vinos_para_NLP_model.csv")

In [21]:
df_descr.head()

,country,description,points,price,province,region_1,taster_name,variety,year,variety_100,designation_20,winery_3
0,France,"Subtle notes of clean, fresh lemon zest promis...",92,39.0,Alsace,Alsace,Anne Krebiehl MW,Riesling,2014,7.0,Pfersigberg Grand Cru,Kuentz-Bas
1,France,"This is a rich wine, labeled sweet but with a ...",93,59.0,Alsace,Alsace,Roger Voss,Gewürztraminer,2012,23.0,Pfersigberg Grand Cru,Kuentz-Bas
2,France,Softly hinted aromatics of dried peach suggest...,90,45.0,Alsace,Alsace,Anne Krebiehl MW,Gewürztraminer,2014,23.0,Pfersigberg Grand Cru,Kuentz-Bas
3,France,"Still very closed up, this is a potentially ri...",91,59.0,Alsace,Alsace,Roger Voss,Gewürztraminer,2013,23.0,Pfersigberg Grand Cru,Kuentz-Bas
4,France,"A delicate, crisp, very pure wine. It has perf...",88,17.0,Alsace,Alsace,Roger Voss,Riesling,2011,7.0,Tradition,Kuentz-Bas


In [22]:
# cantidad de palabras del diccionario que yo le diga
NUM_WORDS = 10000

# largo de cada texto
SEQ_LEN = 256

#creo el tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, oov_token='<UNK>')
tokenizer.fit_on_texts(df_descr['description'])

# convierto los textos a índices numéricos
texto_convert=tokenizer.texts_to_sequences(df_descr['description'])

#trunco el texto si hay más de 256 palabras, como definido en el SEQ_LEN
texto_convert=tf.keras.preprocessing.sequence.pad_sequences(texto_convert, maxlen=SEQ_LEN, padding="post")

print(texto_convert)

[[ 256   28    4 ...    0    0    0]
 [   7    8    5 ...    0    0    0]
 [ 808 3898  746 ...    0    0    0]
 ...
 [ 618   17    4 ...    0    0    0]
 [ 172    4 2144 ...    0    0    0]
 [ 144  148  225 ...    0    0    0]]


In [23]:
len(texto_convert)

5297

In [24]:
### voy a "tokenizar" tambien las etiquetas, o sea las variedades de vino
tipos_vino=pd.DataFrame({'variety': df_descr['variety']})
tipos_vino=tipos_vino.replace(' ', '_', regex=True)

tipos_vino_list = []
for item in tipos_vino['variety']:
    tipos_vino_list.append(str(item))

tipos_vino_tokenizer = tf.keras.preprocessing.text.Tokenizer(split=' ', filters='!"#$%&()*+,./:;<=>?@[\\]^`{|}~\t\n')
tipos_vino_tokenizer.fit_on_texts(tipos_vino_list)


print(tipos_vino_tokenizer.word_index)

tipos_vino_seq = np.array(tipos_vino_tokenizer.texts_to_sequences(tipos_vino_list))
tipos_vino_seq

{'chardonnay': 1, 'pinot_noir': 2, 'cabernet_sauvignon': 3, 'riesling': 4, 'tempranillo': 5, 'malbec': 6, 'syrah': 7, 'merlot': 8, 'red_blend': 9, 'tempranillo_blend': 10, 'gewürztraminer': 11, 'zinfandel': 12, 'pinot_gris': 13, 'rosé': 14, 'sauvignon_blanc': 15, 'bordeaux-style_red_blend': 16, 'cabernet_franc': 17, 'sangiovese': 18, 'viognier': 19, 'nebbiolo': 20, 'pinot_blanc': 21, 'gamay': 22, 'grenache': 23, 'torrontés': 24, 'sparkling_blend': 25, 'petite_sirah': 26, 'petit_verdot': 27, 'champagne_blend': 28, 'white_blend': 29, 'rhône-style_red_blend': 30, 'barbera': 31, 'tinta_de_toro': 32, 'mourvèdre': 33, 'chenin_blanc': 34, 'albariño': 35, 'pinot_grigio': 36, 'garnacha': 37, 'rhône-style_white_blend': 38, 'shiraz': 39, 'mencía': 40, 'bordeaux-style_white_blend': 41, 'melon': 42, 'glera': 43, 'verdejo': 44, 'g-s-m': 45, "nero_d'avola": 46, 'aglianico': 47}


array([[ 4],
       [11],
       [11],
       ...,
       [20],
       [20],
       [20]])

In [25]:
## ahora revierto las indices con los valores, primero con las variedades y luego con la descripción

descr_reverse_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])

def decode_article(text):
    return ' '.join([descr_reverse_index.get(i, '?') for i in text])


# reverse_label_index = dict([(value, key) for (key, value) in label_tokenizer.word_index.items()])

def decode_label(text):
    return ' '.join([tipos_vino_reverse_index.get(i, '?') for i in text])

# reverse_word_index

In [26]:
##enseño las variedades con los índices invertidos
tipos_vino_reverse_index = dict([(value, key) for (key, value) in tipos_vino_tokenizer.word_index.items()])
tipos_vino_reverse_index

{1: 'chardonnay',
 2: 'pinot_noir',
 3: 'cabernet_sauvignon',
 4: 'riesling',
 5: 'tempranillo',
 6: 'malbec',
 7: 'syrah',
 8: 'merlot',
 9: 'red_blend',
 10: 'tempranillo_blend',
 11: 'gewürztraminer',
 12: 'zinfandel',
 13: 'pinot_gris',
 14: 'rosé',
 15: 'sauvignon_blanc',
 16: 'bordeaux-style_red_blend',
 17: 'cabernet_franc',
 18: 'sangiovese',
 19: 'viognier',
 20: 'nebbiolo',
 21: 'pinot_blanc',
 22: 'gamay',
 23: 'grenache',
 24: 'torrontés',
 25: 'sparkling_blend',
 26: 'petite_sirah',
 27: 'petit_verdot',
 28: 'champagne_blend',
 29: 'white_blend',
 30: 'rhône-style_red_blend',
 31: 'barbera',
 32: 'tinta_de_toro',
 33: 'mourvèdre',
 34: 'chenin_blanc',
 35: 'albariño',
 36: 'pinot_grigio',
 37: 'garnacha',
 38: 'rhône-style_white_blend',
 39: 'shiraz',
 40: 'mencía',
 41: 'bordeaux-style_white_blend',
 42: 'melon',
 43: 'glera',
 44: 'verdejo',
 45: 'g-s-m',
 46: "nero_d'avola",
 47: 'aglianico'}

In [27]:
## ahora con la descripción, que aquí ademas ya no tenemos la descripción entera sino cada palabra por separado y tokenizada
descr_reverse_index

{1: '<UNK>',
 2: 'and',
 3: 'the',
 4: 'of',
 5: 'a',
 6: 'with',
 7: 'this',
 8: 'is',
 9: 'flavors',
 10: 'in',
 11: 'wine',
 12: 'on',
 13: 'to',
 14: 'fruit',
 15: 'it',
 16: 'palate',
 17: 'aromas',
 18: 'finish',
 19: 'but',
 20: 'from',
 21: 'are',
 22: 'that',
 23: 'cherry',
 24: "it's",
 25: 'acidity',
 26: 'black',
 27: 'tannins',
 28: 'notes',
 29: 'ripe',
 30: 'oak',
 31: 'nose',
 32: 'drink',
 33: 'by',
 34: 'plum',
 35: 'berry',
 36: 'full',
 37: 'for',
 38: 'red',
 39: 'dry',
 40: 'has',
 41: 'spice',
 42: 'fresh',
 43: 'apple',
 44: 'an',
 45: 'as',
 46: 'bodied',
 47: 'its',
 48: 'light',
 49: 'through',
 50: 'vanilla',
 51: 'while',
 52: 'lemon',
 53: 'well',
 54: 'dark',
 55: 'blackberry',
 56: 'rich',
 57: 'more',
 58: 'texture',
 59: 'raspberry',
 60: 'soft',
 61: 'fruits',
 62: 'shows',
 63: 'now',
 64: 'at',
 65: 'sweet',
 66: 'pear',
 67: 'offers',
 68: 'bright',
 69: 'balanced',
 70: 'juicy',
 71: 'some',
 72: 'not',
 73: 'citrus',
 74: 'crisp',
 75: 'up',
 76:

In [28]:
texto_convert  ## el testo convertido a numeros, será mi "X"

array([[ 256,   28,    4, ...,    0,    0,    0],
       [   7,    8,    5, ...,    0,    0,    0],
       [ 808, 3898,  746, ...,    0,    0,    0],
       ...,
       [ 618,   17,    4, ...,    0,    0,    0],
       [ 172,    4, 2144, ...,    0,    0,    0],
       [ 144,  148,  225, ...,    0,    0,    0]])

In [29]:
tipos_vino_seq  ## las variedades convertidas a numero, será mi target

array([[ 4],
       [11],
       [11],
       ...,
       [20],
       [20],
       [20]])

In [30]:
## divino en train y test


X_train, X_test, y_train, y_test = train_test_split(texto_convert,
                                                    tipos_vino_seq,
                                                    test_size=0.20,
                                                    random_state=42)

print('Test: ' + str(len(X_test)) + ' Train: ' + str(len(X_train)))

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
y_train = y_train.astype("float32")
y_test = y_test.astype("float32")


print( "X_train",X_train.shape)
print("y_train",y_train.shape)

Test: 1060 Train: 4237
X_train (4237, 256)
y_train (4237, 1)


In [31]:
### creo el modelo 

EMBEDDING_SIZE = 256
EMBEDDING_SIZE_2 = 64
EMBEDDING_SIZE_3 = 48  ### las 47 variedades + 1
BATCH_SIZE = 30  
EPOCHS = 10


model = tf.keras.Sequential([
    
    
    tf.keras.layers.Embedding(NUM_WORDS, EMBEDDING_SIZE),
    tf.keras.layers.Conv1D(128, 5, activation='relu'), 
    tf.keras.layers.GlobalMaxPooling1D(), 
    tf.keras.layers.Dense(EMBEDDING_SIZE_2, activation='relu'),
    tf.keras.layers.Dense(EMBEDDING_SIZE_3, activation='softmax')
])

model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 256)         2560000   
                                                                 
 conv1d_2 (Conv1D)           (None, None, 128)         163968    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 48)                3120      
                                                                 
Total params: 2,735,344
Trainable params: 2,735,344
Non-trainable params: 0
____________________________________________

In [32]:
## lo ejecuto

history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_test, y_test),
                    validation_steps=30
                   )


loss, accuracy = model.evaluate(X_test, y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/10
142/142 [==============================] - 10s 67ms/step - loss: 3.0623 - accuracy: 0.1990 - val_loss: 2.6370 - val_accuracy: 0.3611
Epoch 2/10
142/142 [==============================] - 10s 71ms/step - loss: 2.1381 - accuracy: 0.4312 - val_loss: 2.1099 - val_accuracy: 0.4389
Epoch 3/10
142/142 [==============================] - 10s 68ms/step - loss: 1.5091 - accuracy: 0.5839 - val_loss: 1.9141 - val_accuracy: 0.4778
Epoch 4/10
142/142 [==============================] - 10s 68ms/step - loss: 0.9415 - accuracy: 0.7494 - val_loss: 1.8114 - val_accuracy: 0.5056
Epoch 5/10
142/142 [==============================] - 10s 68ms/step - loss: 0.4943 - accuracy: 0.8886 - val_loss: 1.8046 - val_accuracy: 0.4933
Epoch 6/10
142/142 [==============================] - 10s 68ms/step - loss: 0.2197 - accuracy: 0.9681 - val_loss: 1.8756 - val_accuracy: 0.5122
Epoch 7/10
142/142 [==============================] - 10s 68ms/step - loss: 0.0903 - accuracy: 0.9943 - val_loss: 1.9833 - val_accuracy:

In [33]:
### Después de toquetear un poco los parámetros he conseguido un accuracy del 51%, no es tan catastrófico como las primeras pruebas que hice
## pero tampoco está mejorando el modelo de regresión logística, que fue él que me dió hasta ahora los mejores resultados.

### Finalmente, abandono este camino y me quedo con el mejor score obtenido hasta ahora: 57,67% (casi un 58%)

## Las varias pruebas de cambian parámetros las he hecho, por rapidéz, sobre este mismo notebook y, viendo que no conseguía subir del 51% he decidido
## no seguir, por falta de tiempo basicamente, estando a apenas unos días de la entrega

## No descarto retomar este enfoque más adelante y, quizás, poder mejorar un poco los resultados

